In [1]:
import matplotlib.pyplot as plt
from ipywidgets import interactive, Layout, HBox, FloatSlider
import numpy as np
import math

%config InlineBackend.figure_format = 'retina'

**Generalised logistic function (GLF)**

![y = A + \frac{K - A}{\left( C + Q \cdot e^{-By} \right)^{ 1 / \nu} }](assets/GLF.png)

taken from https://en.wikipedia.org/wiki/Generalised_logistic_function

**Inverse GLF**
The following equation is the one used in this simulation.

![y = \frac{1}{B} \cdot ln \left( \frac{ \left( \frac{K - A}{x - A} \right)^\nu -C }{Q} \right) + E](assets/GLF_inverse.png)

A bunch of parameter can be removed, as they are not used:
- `A` is the minimum value of the x axis, which will always be `0`
- `C` and `Q` are form parameters that are meant to be set to `1` by default

With those parameters replaced by their default values, here is the simplified equation used:

![y = \frac{1}{B} \cdot ln \left( \left( \frac{K}{x} \right)^\nu -1 \right) + E](assets/GLF_inverse_simplified.png)


In [2]:
class Param:
    def __init__(self, name, value, min, max, step, description):
        self.name = name
        self.value = value
        self.min = min
        self.max = max
        self.step = step
        self.description = description

    def slider(self):
        style = {'description_width': 'initial', 'value_width': 'initial'}

        return FloatSlider(min=self.min, max=self.max, step=self.step, value=self.value, description=f"{self.description} ({self.name})", style=style)
    
class Params:
    def __init__(self, *args: list[Param]):
        for arg in args:
            if not isinstance(arg, Param):
                raise ValueError("All arguments must be of type Param")
            
            setattr(self, arg.name, arg)
        
def equation(x, A, B, C, K, Q, nu, E):
    try:
        upper = math.pow((K-A)/(x-A), nu) - C
        return (1/B) * math.log(upper / Q) + E
    except: 
        return None
    
def equation2(x, B, K, nu, E):
    try:
        inner = math.pow((K/x), nu) - 1
        return (1/B) * math.log(inner) + E        
    except: 
        return None
    
# def plot_static(A, B, C, K, nu, Q, E):
def plot_static(B, K, nu, E):
    x_range = np.linspace(0, 1, 1000000)[1:-1]
    # y_range = [equation(x, A, B, C, K, Q, nu, E) for x in x_range]   
    y_range2 = [equation2(x, B, K, nu, E) for x in x_range]   

    # plt.plot(x_range, y_range, 'g')
    plt.plot(x_range, y_range2, 'g')
    plt.axhline(y=0, color='black')
    plt.xlim(-0.1, 1.1)
    plt.ylim(-5, 20)
    plt.xlabel("Any ratio")
    plt.ylabel("APR (%)")
    plt.grid()

def plot_interactive(params: Params):
    sliders = {param.name: param.slider() for param in vars(params).values()}

    widgets = interactive(plot_static, **sliders)

    controls = HBox(widgets.children[:-1], layout = Layout(flex_flow='row wrap'))

    display(controls)
    display(widgets.children[-1])

In [3]:
params = Params(
    Param("B", 1.25, 0, 10, 0.01, "Flatness"),
    Param("E", 11.5, 0, 20, 0.1, "V. Offset"),
    Param("K", 0.5, 0, 1, 0.01, "Maximum ratio"),
    Param("nu",1.5, 0, 2, 0.01, "Skewness"),
    # Param("A", 0, 0, 0.5, 0.01, "Minimum ratio (implicit)"),
    # Param("C", 1, 0, 2, 0.01, "(meaningless)"),
    # Param("Q", 1, -10, 10, 0.1, "(meaningless)"),
)

plot_interactive(params)

Output()